<a href="https://colab.research.google.com/github/dgiri4132/pairs_trading_strategy-zscore/blob/main/Pairs_trading_Stratedy_using_zscore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd

option_for_index={'S&P500':'^GSPC','NASDAQ-100':'^NDX','Dow Jones':'^DJI','FTSE 100':'FTSE','Nikkei 225':'^N225','DAX':'^GDAXI','CAC 40':'^FCHI'}
chosen_index=input(f'Choose anyone of the index from indexes below:{option_for_index.keys()} ').strip()
def index_tickers(chosen_index):
  try:
    if chosen_index=="S&P500":
      df = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
      return df['Symbol'].tolist()
    elif chosen_index=='NASDAQ-100':
      df=pd.read_html('https://en.wikipedia.org/wiki/NASDAQ-100')[3]
      return df['Symbol'].tolist()
    elif chosen_index=='Dow Jones':
      df=pd.read_html('https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average')[1]
      return df['Symbol'].tolist()
    elif chosen_index=='FTSE 100':
      df=pd.read_html('https://en.wikipedia.org/wiki/FTSE_100_Index')[3]
      return df['Symbol'].tolist()
    elif chosen_index=='Nikkei 225':
      df=pd.read_html('https://en.wikipedia.org/wiki/Nikkei_225')[1]
      return df['Symbol'].tolist()
    elif chosen_index=='DAX':
      df=pd.read_html('https://en.wikipedia.org/wiki/DAX')[1]
      return df['Symbol'].tolist()
    elif chosen_index=='CAC 40':
      df=pd.read_html('https://en.wikipedia.org/wiki/CAC_40')[0]
      return df['Symbol'].tolist()
    else:
      print('Index not recognized or supported')
      return None
  except Exception as e:
    print(f'Error loading tickers for {chosen_index}:{e}')
    return None




In [ ]:
!git init
!git config --global user.email "darshangiri785@gmail.com"
!git config --global user.name "dgiri4132"


Reinitialized existing Git repository in /content/.git/


In [ ]:
!git remote add origin https://github.com/dgiri4132/pairs_trading_strategy-zscore.git
!git branch -M main
!git add .
!git commit -m "Initial commit for pairs trading strategy using z-score"
!git push -u origin main


[main (root-commit) 9aa3adc] Initial commit for pairs trading strategy using z-score
 21 files changed, 51025 insertions(+)
 create mode 100644 .config/.last_opt_in_prompt.yaml
 create mode 100644 .config/.last_survey_prompt.yaml
 create mode 100644 .config/.last_update_check.json
 create mode 100644 .config/active_config
 create mode 100644 .config/config_sentinel
 create mode 100644 .config/configurations/config_default
 create mode 100644 .config/default_configs.db
 create mode 100644 .config/gce
 create mode 100644 .config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db
 create mode 100644 .config/logs/2025.06.11/13.36.48.453813.log
 create mode 100644 .config/logs/2025.06.11/13.37.08.919149.log
 create mode 100644 .config/logs/2025.06.11/13.37.17.609931.log
 create mode 100644 .config/logs/2025.06.11/13.37.18.763091.log
 create mode 100644 .config/logs/2025.06.11/13.37.27.125319.log
 create mode 100644 .config/logs/2025.06.11/13.37.27.755669.log
 create mode 100755 